## CSV -> JSON 변환 코드

In [1]:
import csv
import json

def csv_to_json(csv_file, json_file):
    # CSV 파일 읽기
    with open(csv_file, mode='r', encoding='utf-8') as file:
        csv_reader = csv.DictReader(file)
        data = [row for row in csv_reader]  # 각 행을 딕셔너리로 저장
        
    # JSON 파일 쓰기
    with open(json_file, mode='w', encoding='utf-8') as file:
        json.dump(data, file, indent=4, ensure_ascii=False)  # ensure_ascii=False로 한글 지원

    print(f"CSV 파일이 JSON으로 변환되었습니다: {json_file}")

In [11]:
import pandas as pd

df_train = pd.read_csv('./resources/raw/train_reformat.csv')

In [12]:
df_train_head5 = df_train.iloc[:5]

In [13]:
df_train_head5.to_csv('train_reformat_head5.csv')

In [14]:
csv_to_json('./resources/head5/train_reformat_head5.csv', './resources/data_aug/train_reformat_head5.json')

CSV 파일이 JSON으로 변환되었습니다: ./resources/data_aug/train_reformat_head5.json


### `\n` 처리 확인

In [18]:
# 데이터프레임에서 특정 행을 JSON으로 저장
def save_row_as_json(df, row_condition, json_file):
    # 조건에 맞는 행 필터링 (예: id가 특정 값인 경우)
    filtered_row = df[row_condition]

    # JSON으로 저장 (한 행만 포함되도록 `to_dict` 사용)
    if not filtered_row.empty:
        # `to_dict`에서 `orient='records'`를 사용하여 딕셔너리 리스트로 변환
        data = filtered_row.to_dict(orient='records')[0]  # 첫 번째 딕셔너리만 추출
        with open(json_file, 'w', encoding='utf-8') as file:
            import json
            json.dump(data, file, indent=4, ensure_ascii=False)  # ensure_ascii=False로 한글 처리
        print(f"JSON 파일이 저장되었습니다: {json_file}")
    else:
        print("조건에 맞는 행이 없습니다.")

# 행을 JSON 파일로 저장
save_row_as_json(df_train, df_train['id'] == 'generation-for-nlp-520', 'df_520.json')

JSON 파일이 저장되었습니다: df_520.json


## JSON -> CSV 변환 코드

In [50]:
import csv
import json

def json_to_csv(json_file, csv_file):
    # JSON 파일 읽기
    with open(json_file, mode='r', encoding='utf-8') as file:
        data = json.load(file)  # JSON 데이터를 Python 리스트/딕셔너리로 변환

    # CSV 파일 쓰기
    with open(csv_file, mode='w', encoding='utf-8', newline='') as file:
        # JSON 데이터를 기반으로 CSV 필드명 추출
        if data:
            fieldnames = data[0].keys()  # 첫 번째 딕셔너리의 키를 필드명으로 사용
            csv_writer = csv.DictWriter(file, fieldnames=fieldnames)

            csv_writer.writeheader()  # 헤더 작성
            csv_writer.writerows(data)  # 데이터 작성

    print(f"JSON 파일이 CSV로 변환되었습니다: {csv_file}")

In [51]:
json_to_csv('kbs_korean_data.json', 'kbs_korean_data.csv')

JSON 파일이 CSV로 변환되었습니다: kbs_korean_data.csv


## 54회 시험지 PDFMiner

In [19]:
import json
import re

def clean_text(text):
    # 이중 띄어쓰기를 단일 띄어쓰기로 변경
    text = re.sub(r'\s+', ' ', text)
    # 불필요한 개행, 페이지 번호, 시험 정보 제거
    text = re.sub(r'-\s*\d+\s*-.*$', '', text, flags=re.MULTILINE)
    text = text.strip()
    return text

def convert_to_json(text):
    # 문제 구분 패턴
    problems = re.split(r'\n(\d+\.)', text)[1:]
    
    json_data = []
    
    for i in range(0, len(problems), 2):
        problem_number = problems[i].rstrip('.')
        problem_content = problems[i+1].strip()
        
        # 질문 추출
        question_match = re.search(r'(.*?)(\n[①②③④⑤].*)', problem_content, re.DOTALL)
        if question_match:
            question = clean_text(question_match.group(1))
            choices_text = question_match.group(2)
        else:
            question = ''
            choices_text = problem_content
        
        # 보기 추출
        choices = re.findall(r'[①②③④⑤]\s*(.*?)(?=[①②③④⑤]|$)', choices_text, re.DOTALL)
        choices = [clean_text(choice) for choice in choices]

        problem_entry = {
            "id": f"kbs-korean54-{problem_number}",
            "paragraph": "",
            "question": question,
            "choices": str(choices),
            "answer": "",
            "question_plus": ""
        }
        
        json_data.append(problem_entry)
    
    return json_data

In [20]:
with open('kbs_korean_54th.txt', 'r', encoding='utf-8') as file:
    text = file.read()

result = convert_to_json(text)

with open('temp.json', 'w', encoding='utf-8') as file:
    json.dump(result, file, ensure_ascii=False, indent=4)

In [14]:
import re

def clean_text(text):
    # 페이지 번호와 시험 정보 제거
    text = re.sub(r'-\s*\d+\s*-\n.*?KBS.*?시험\(.*?\)', '', text, flags=re.DOTALL)
    
    # 이중 이상 띄어쓰기를 단일 띄어쓰기로 변경
    text = re.sub(r'\s+', ' ', text)
    
    # 엔터로 구분된 문단을 줄글로 변경
    text = re.sub(r'\n+', ' ', text)
    
    return text.strip()

In [15]:
text = '''

    어느  공공  주택  단지의  인적이  드문  공터에  사람들이  버린  쓰레기가  쌓여  해당  구청  측이  골머리를 

앓고  있었다.  주기적으로  청소차와  인력을  보내  깔끔하게  치워도  하루  이틀만  지나면  다시  쓰레기가 

쌓였다.  해결  방법을  고심하던  차에  한  공무원이  아이디어를  냈고,  그  결과  공터에  예쁜  꽃밭이  조성

되었다.  그  이후  더  이상  그곳에  쓰레기가  쌓이지  않았다.  어떻게  이런  변화가  일어난  것일까?
    어떤  선택을  강제로  금지하거나  경제적  유인책을  제공하지  않고도  예상  가능한  방향으로  사람들의 

행동을  변화하도록  유도하는  것을  ‘넛지(nudge)’라고  한다.  ‘넛지’는  원래  ‘팔꿈치로  슬쩍  찌르다’라는 

의미를  지닌  영어  단어지만,  미국의  행동경제학자  리처드  탈러와  하버드  로스쿨의  캐스  선스타인  교

수는  ‘사람들의  바람직한  선택을  부드럽게  유도하되,  선택의  자유는  여전히  개인에게  열려  있는  상태’

라는  의미로  이를  새롭게  정의했다.  ‘넛지’  형태의  간섭은  쉽게  피할  수  있어야  하며,  또  그렇게  하는 

데  비용도  적게  들어야  한다.  또한  ‘넛지’는  명령이나  지시가  아니다.  쓰레기  불법  투기를  막기  위해 
공터에  꽃밭을  조성하는  것은  ‘넛지’다.  그러나  쓰레기  불법  투기자를  CCTV로  촬영하여  벌금을  부과

하는  등의  법적  제재를  가하는  것은  ‘넛지’가  아니다.

    이러한  ‘넛지’를  만드는  사람을  ‘선택  설계자(choice  architect)’라  부른다.  ‘선택  설계자’는  사람들이 

결정을  내리는  데  배경이  되는  정황이나  맥락을  만드는  사람이다.  우리  주변에는  이러한  ‘선택  설계

자’가  무수히  많다.  환자에게  선택  가능한  다양한  치료법을  설명하고  소개해야  하는  의사도  ‘선택  설

계자’다.  학생에게  실천  가능한  여러  공부  방법이나  학교생활  개선  방법  등을  안내하는  교사도  ‘선택 
설계자’다.

-  20  -

                                                                                        제54회  KBS  한국어능력시험(홀수형)

    리처드  탈러와  캐스  선스타인  교수는  ‘선택  설계자’에  의한  ‘넛지’가  필요한  이유를  인간의  성향  때

문이라고  설명한다.  그들은  인간이  불합리한  존재라고  주장한다.  인간들  스스로는  자신을  합리적인  존

재라고  여기고  있지만,  부족한  정보나  잘못된  편견,  판단  등으로  인해  부적절한  결정을  내리는  경우가 

많다는  것이다.  예를  들어,  어느  백화점에서  당일  20만  원  이상  구매  고객에게  1만원  상당의  사은품

을  주는  행사를  실시하는  경우,  많은  사람들이  사은품을  받기  위해  필요하지도  않은  물건을  구매한다. 

사은품을  받은  사람들은  자신이  합리적  선택을  했다고  흐뭇해한다.  하지만  약간의  시간이  지나  자신
이  구입한  필요  없는  물건을  보고는  이내  자신의  잘못된  선택에  후회를  하는  것이다.  저자들은  이러

한  인간의  불완전성에  근거하여  ‘넛지’의  필요성을  역설한다.  어떤  특정한  정책이나  방침이  보다  나은 

결과를  가져올  것이라  예측될  경우  ‘넛지’를  이용해  선택의  자유를  존중하면서도  좀  더  현명한  선택을 

하게끔  사람들을  이끌어야  한다는  것이다.

    물론,  이러한  ‘넛지’에  반대하는  사람들도  존재한다.  선택의  자유를  옹호하는  사람들은  그  누구도  사

람들의  선택을  의도적으로  유도하거나  조종해서는  안  된다고  주장한다.  그들은  ‘넛지’의  필요성을  주
장하는  사람들이  매우  미끄러운  비탈길을  미끄러져  내려가기  시작했다고  우려한다.  즉,  자유주의적  개

입주의를  허용하고  받아들이면  이후에는  극도의  개입주의적  간섭이  뒤따를  것이  분명하다는  것이다. 

그러나  ‘넛지’  찬성론자들은  과도한  자유주의가  오히려  인간의  잘못된  선택을  방관하고,  그  결과  심각

한  위험을  초래할  수  있다는  점을  지적하며,  자유주의적  개입주의인  ‘넛지’의  필요성을  강조하고  있다.
'''

In [16]:
cleaned_text = clean_text(text)
print(cleaned_text)

어느 공공 주택 단지의 인적이 드문 공터에 사람들이 버린 쓰레기가 쌓여 해당 구청 측이 골머리를 앓고 있었다. 주기적으로 청소차와 인력을 보내 깔끔하게 치워도 하루 이틀만 지나면 다시 쓰레기가 쌓였다. 해결 방법을 고심하던 차에 한 공무원이 아이디어를 냈고, 그 결과 공터에 예쁜 꽃밭이 조성 되었다. 그 이후 더 이상 그곳에 쓰레기가 쌓이지 않았다. 어떻게 이런 변화가 일어난 것일까? 어떤 선택을 강제로 금지하거나 경제적 유인책을 제공하지 않고도 예상 가능한 방향으로 사람들의 행동을 변화하도록 유도하는 것을 ‘넛지(nudge)’라고 한다. ‘넛지’는 원래 ‘팔꿈치로 슬쩍 찌르다’라는 의미를 지닌 영어 단어지만, 미국의 행동경제학자 리처드 탈러와 하버드 로스쿨의 캐스 선스타인 교 수는 ‘사람들의 바람직한 선택을 부드럽게 유도하되, 선택의 자유는 여전히 개인에게 열려 있는 상태’ 라는 의미로 이를 새롭게 정의했다. ‘넛지’ 형태의 간섭은 쉽게 피할 수 있어야 하며, 또 그렇게 하는 데 비용도 적게 들어야 한다. 또한 ‘넛지’는 명령이나 지시가 아니다. 쓰레기 불법 투기를 막기 위해 공터에 꽃밭을 조성하는 것은 ‘넛지’다. 그러나 쓰레기 불법 투기자를 CCTV로 촬영하여 벌금을 부과 하는 등의 법적 제재를 가하는 것은 ‘넛지’가 아니다. 이러한 ‘넛지’를 만드는 사람을 ‘선택 설계자(choice architect)’라 부른다. ‘선택 설계자’는 사람들이 결정을 내리는 데 배경이 되는 정황이나 맥락을 만드는 사람이다. 우리 주변에는 이러한 ‘선택 설계 자’가 무수히 많다. 환자에게 선택 가능한 다양한 치료법을 설명하고 소개해야 하는 의사도 ‘선택 설 계자’다. 학생에게 실천 가능한 여러 공부 방법이나 학교생활 개선 방법 등을 안내하는 교사도 ‘선택 설계자’다. 리처드 탈러와 캐스 선스타인 교수는 ‘선택 설계자’에 의한 ‘넛지’가 필요한 이유를 인간의 성향 때 문이라고 설명한다. 그들은 인간이 불합리한 존재라고 주장한다. 인간들 스스로는 자신을 합리적인 존 재라고 여기

In [23]:
text = '''

백제와  신라의  황산벌  전투(660년)를  현대적  시각에서  해석한  영화  <황산벌>에서  백제인은  전라도 
사투리를,  신라인은  경상도  사투리를  각각  사용하고  있다.  영화  ‘황산벌’에는  백제  군영에  몰래  잠입
한  신라  첩자가  계백  장군의  말을  전하는  장면이  나온다.

 

첩자 :  백제  계백  장군이  “병사들에게  거시기하게  하고  머시기가  거시기할  때까지  거시기해  부러

라.”라고  했습니다.

김유신:  암호  해독관!  해석해  봐라.   
암호  해독관 :  첫  번째  거시기는  경계를  철저히  하라는  의미로  쉽게  해석되지만,  나머지  머시기와 

거시기는  죽어도  모르겠습니다.   

많은  사람들이  ‘거시기’를  전라도  방언쯤으로  알고  있으나,  ‘거시기’는  표준어이며  ‘머시기’가  ‘거시
기’의  전라도  방언이다.  거시기는  대명사와  감탄사로  쓰인다.  대명사  ‘거시기’는  말하는  도중에  사람이
나  사물의  이름이  얼른  생각나지  않거나  바로  말하기  곤란할  때,  그  이름  대신으로  내는  말이다.  감탄
사  거시기는  하려는  말이  얼른  생각나지  않거나  바로  말하기가  거북할  때  쓰는  군소리에  해당한다.   

발화(發話)는  생각이  음성적으로  실현된  것을  말하며,  이러한  발화들이  모여서  이루어진  유기적인 
통일체를  담화(談話)라고  한다.  ‘거시기’와  ‘머시기’가  지칭하는  것을  파악하기  위해서는  발화만을  분
석해서는  곤란하며,  지칭  표현이  사용된  담화의  구조를  파악해야  한다.  이를  상황  의존적이라고  하며, 
담화의  구조를  파악하는  데는  언어적  표현  못지않게  장면도  중요한  역할을  한다.  이처럼  발화는  말하
는  이,  듣는  이,  장면에  따라  의미가  결정된다.  담화의  구성  요소  네  가지를  말하는  이,  듣는  이,  내용
(발화),  장면이라고  하는  까닭이  여기에  있다. 
    장면은  언어  외적  장면과  언어  내적  장면으로  구분할  수  있다.  언어  외적  장면은  말하는  이와  듣는 
이가  있는  시간과  공간이다.  그리고  언어  내적  장면은  한  발화의  앞뒤에  이어지는  발화의  연속체로서, 
담화의  흐름이나  의미  해석에  결정적인  역할을  한다.  백제  계백  장군이  사용한  대명사  ‘거시기’는  말
하는  이와  듣는  이가  이미  알고  있는  어떤  것을  가리킨다.  신라군이  계백  장군의  말을  이해하지  못 
하는  까닭은              ㉠              이다.  쉽게  말해  앞뒤  발화  정보가  없기  때문이다. 

언어  외적  장면과  언어  내적  장면  외에  담화  공동체의  가치,  역사적  경험,  관습과  같은  사회·문화
적  장면도  있다.  장면을  맥락이라고  표현하기도  하는데,  언어  외적  장면과  언어  내적  장면을  아울러서 
상황  맥락이라고  한다.  사회·문화적  맥락은  문화권에  따라  다르기  때문에  외국인과  의사소통할  때는 
특히  주의해야  한다.  이처럼  장면(맥락)은  담화에  구체적인  의미를  부여하는  중요한  역할을  한다. 
'''

In [24]:
cleaned_text = clean_text(text)
print(cleaned_text)

백제와 신라의 황산벌 전투(660년)를 현대적 시각에서 해석한 영화 <황산벌>에서 백제인은 전라도 사투리를, 신라인은 경상도 사투리를 각각 사용하고 있다. 영화 ‘황산벌’에는 백제 군영에 몰래 잠입 한 신라 첩자가 계백 장군의 말을 전하는 장면이 나온다. 첩자 : 백제 계백 장군이 “병사들에게 거시기하게 하고 머시기가 거시기할 때까지 거시기해 부러 라.”라고 했습니다. 김유신: 암호 해독관! 해석해 봐라. 암호 해독관 : 첫 번째 거시기는 경계를 철저히 하라는 의미로 쉽게 해석되지만, 나머지 머시기와 거시기는 죽어도 모르겠습니다. 많은 사람들이 ‘거시기’를 전라도 방언쯤으로 알고 있으나, ‘거시기’는 표준어이며 ‘머시기’가 ‘거시 기’의 전라도 방언이다. 거시기는 대명사와 감탄사로 쓰인다. 대명사 ‘거시기’는 말하는 도중에 사람이 나 사물의 이름이 얼른 생각나지 않거나 바로 말하기 곤란할 때, 그 이름 대신으로 내는 말이다. 감탄 사 거시기는 하려는 말이 얼른 생각나지 않거나 바로 말하기가 거북할 때 쓰는 군소리에 해당한다. 발화(發話)는 생각이 음성적으로 실현된 것을 말하며, 이러한 발화들이 모여서 이루어진 유기적인 통일체를 담화(談話)라고 한다. ‘거시기’와 ‘머시기’가 지칭하는 것을 파악하기 위해서는 발화만을 분 석해서는 곤란하며, 지칭 표현이 사용된 담화의 구조를 파악해야 한다. 이를 상황 의존적이라고 하며, 담화의 구조를 파악하는 데는 언어적 표현 못지않게 장면도 중요한 역할을 한다. 이처럼 발화는 말하 는 이, 듣는 이, 장면에 따라 의미가 결정된다. 담화의 구성 요소 네 가지를 말하는 이, 듣는 이, 내용 (발화), 장면이라고 하는 까닭이 여기에 있다. 장면은 언어 외적 장면과 언어 내적 장면으로 구분할 수 있다. 언어 외적 장면은 말하는 이와 듣는 이가 있는 시간과 공간이다. 그리고 언어 내적 장면은 한 발화의 앞뒤에 이어지는 발화의 연속체로서, 담화의 흐름이나 의미 해석에 결정적인 역할을 한다. 백제 계백 장군이 사용한 대명사 ‘거시기’는 말 

In [25]:
text = '''
인간처럼  유성생식을  하는  생물들은  모두  난자와  정자가  결합하는  수정이라는  과정을  거쳐  태어

난다.  암컷과  수컷이  각각  자기  유전자의  절반을  넣어  만든  난자와  정자가  만나  하나의  수정란이  되

어야  그로부터  새로운  생명체가  탄생하는  것이다.  우리가  흔히  유전자라고  부르는  것들은  대개  한데 

뭉뚱그려  세포의  핵  속에  들어  있는  DNA를  의미한다.  그러나  세포  안에는  핵뿐  아니라  많은  세포소

기관들이  들어  있다.  그  중의  하나로  세포가  사용하는  에너지를  만들어  내는  미토콘드리아라는  소기

관이  있다.  그런데  신기하게도  이  미토콘드리아  안에는  핵의  DNA와  다른  그들만의  고유한  DNA가 

들어  있다.  이는  그  옛날  세포가  진화하던  초창기에  미토콘드리아가  독립적으로  생활하던  박테리아

였다는  결정적인  증거인  셈이다.  이른바  ‘공생설’이라고  부르는  진화생물학  이론은  서로  다른  박테리

아들이  공생과정을  통해  오늘날의  세포를  형성하게  되었다고  설명한다.   

따라서  핵이  융합하는  과정에서는  당연히  암수의  유전자가  공평하게  절반씩  결합하지만  핵을  제

외한  세포질은  암컷이  홀로  제공하는  것이기  때문에  미토콘드리아의  DNA는  온전히  암컷으로부터 

오게  된다.  바로  이런  이유  때문에  생물의  계통을  밝히는  연구에서는  미토콘드리아의  DNA를  비교 

분석하게  된다.  즉  철저하게  암컷의  계보를  거슬러  올라가는  것이다.  전통적으로  ㉠ 남자만  이름을 

올릴  수  있는  우리  족보와는  달리  생물학적인  족보는  암컷  즉  여성의  혈통만을  기록한다. 

수정과  발생의  과정에서  남성이  주도권을  쥐어야  한다는  강박관념  때문에  만들어진  억지스러운 

일들이  인간  사회에는  심심찮게  존재한다.  17∼18세기  유럽의  생물학자들도  예외가  아니었다.  DNA

의  존재를  모르던  시절이긴  하지만  당시  생물학자들은  정자  안에  이미  작은  인간이  들어앉아  있다고 

주장했다.  ‘씨’는  이미  남성에  의해  결정되어  있고  이름하여  ‘씨받이’로  간주된  여성은  그저  영양분을 

제공하여  씨를  싹  틔우는  밭에  불과하다고  설명하려  했다.  정자  속에  이미  작은  사람이  들어  있다는 

이론을  받아들이면  실로  어처구니없는  모순에  빠질  수밖에  없었다.  마치  러시아의  전통  인형처럼  그 

작은  사람의  정자  속에는  더  작은  사람이  웅크리고  있어야  하고,  또  그  사람의  정자  속에는  더  작은 

사람이  있어야  하고,  그  사람의  정자  속에  또  더  작은  사람이  들어  있어야  하는  식의  ㉡ 무한대의  모

순을  범할  수밖에  없는  것이다.  그릇된  이념은  결국  과학의  객관성  앞에  무너지게  되어  있다. 

수정  과정에서  암수의  역할은  다분히  비대칭적이다.  정자는  수컷의  유전물질을  난자에  전달하고 

나면  그  소임을  다하지만  난자는  암컷의  유전물질은  물론  생명체의  초기  발생에  필요한  온갖  영양분

을  다  갖추고  있어야  한다.  핵  DNA는  정확하게  반씩  투자하지만  미토콘드리아  등  다른  세포소기관

의  DNA는  암컷만이  홀로  제공하므로  유전물질만  비교해도  암컷의  기여도가  더  크다고  봐야  한다. 

많은  경우  유전물질이  일단  배달된  다음에  벌어지는  일에  대해서는  전혀  아는  바도  없는  수컷이  훗

날  뒤늦게  정통성을  주장하는  것은  생물학적으로  어불성설이다.
'''

In [26]:
cleaned_text = clean_text(text)
print(cleaned_text)

인간처럼 유성생식을 하는 생물들은 모두 난자와 정자가 결합하는 수정이라는 과정을 거쳐 태어 난다. 암컷과 수컷이 각각 자기 유전자의 절반을 넣어 만든 난자와 정자가 만나 하나의 수정란이 되 어야 그로부터 새로운 생명체가 탄생하는 것이다. 우리가 흔히 유전자라고 부르는 것들은 대개 한데 뭉뚱그려 세포의 핵 속에 들어 있는 DNA를 의미한다. 그러나 세포 안에는 핵뿐 아니라 많은 세포소 기관들이 들어 있다. 그 중의 하나로 세포가 사용하는 에너지를 만들어 내는 미토콘드리아라는 소기 관이 있다. 그런데 신기하게도 이 미토콘드리아 안에는 핵의 DNA와 다른 그들만의 고유한 DNA가 들어 있다. 이는 그 옛날 세포가 진화하던 초창기에 미토콘드리아가 독립적으로 생활하던 박테리아 였다는 결정적인 증거인 셈이다. 이른바 ‘공생설’이라고 부르는 진화생물학 이론은 서로 다른 박테리 아들이 공생과정을 통해 오늘날의 세포를 형성하게 되었다고 설명한다. 따라서 핵이 융합하는 과정에서는 당연히 암수의 유전자가 공평하게 절반씩 결합하지만 핵을 제 외한 세포질은 암컷이 홀로 제공하는 것이기 때문에 미토콘드리아의 DNA는 온전히 암컷으로부터 오게 된다. 바로 이런 이유 때문에 생물의 계통을 밝히는 연구에서는 미토콘드리아의 DNA를 비교 분석하게 된다. 즉 철저하게 암컷의 계보를 거슬러 올라가는 것이다. 전통적으로 ㉠ 남자만 이름을 올릴 수 있는 우리 족보와는 달리 생물학적인 족보는 암컷 즉 여성의 혈통만을 기록한다. 수정과 발생의 과정에서 남성이 주도권을 쥐어야 한다는 강박관념 때문에 만들어진 억지스러운 일들이 인간 사회에는 심심찮게 존재한다. 17∼18세기 유럽의 생물학자들도 예외가 아니었다. DNA 의 존재를 모르던 시절이긴 하지만 당시 생물학자들은 정자 안에 이미 작은 인간이 들어앉아 있다고 주장했다. ‘씨’는 이미 남성에 의해 결정되어 있고 이름하여 ‘씨받이’로 간주된 여성은 그저 영양분을 제공하여 씨를 싹 틔우는 밭에 불과하다고 설명하려 했다. 정자 속에 이미 작은 사람이 들어 있

In [27]:
text = '''
    예술과  경제는  서로  다른  힘에  의해  움직이는  전혀  다른  세계일까?  그렇지  않다.  예술과  경제를  움

직이는  공통된  힘이  존재한다.  이  두  분야를  움직이는  힘이  같다면  각  분야가  작동하는  원리와  문제

를  해결하는  방법도  비슷할  것이다.  뛰어난  예술가들이  걸어  온  길을  살펴  우리의  경제와  기업  경영

에  활용해야  하는  이유가  바로  여기에  있다.

    훌륭한  예술가는  이전의  상식을  허물고  새롭게  정의하는  재정의력을  활용해  독창적인  작품을  내놓

는다.  “미술의  역사는  미술에  대한  재정의의  역사다.”라고  할  만큼  예술의  대가들은  나름대로  미술을 

재정의하며  창작해  왔다.  관습적  결합과  익숙한  인과관계를  해체하고  새롭게  합성하는  창조적  융합을 

통해  재정의력을  보여준  화가가  바로  초현실주의  화가  르네  마그리트다.  그의  <공동  발명>이란  작품

을  보자.  우리가  그동안  익숙하게  알고  있던  상식적인  인어와는  반대로  상반신이  어류이고  하반신이 

사람이다.  이  그림을  통해  마그리트는  ‘거꾸로  뒤바꾸기’를  활용한  새로운  융합을  통해  인어를  재정의

했다.  너무나  익숙한  현실을  거꾸로  뒤집어  지극히  사실적으로  표현하는  것이  초현실주의  화가  마그

리트가  재정의한  미술이었다.

   

공동  발명 

얼핏  보면  무섭지만  실은  좋은  사람이다

    금융에서도  신상품을  개발하는데  ‘거꾸로  뒤바꾸기’를  통한  재정의력이  활용된  사례가  있다.  대표적 

상품이  바로  역모기지대출이다.  역모기지대출이란  고령자가  자신의  집을  담보로  노후  생활  자금을  연

금  형태로  지급받고,  그  고령자가  사망한  후  은행이  주택을  처분하여  고령자에게  지급한  원금을  상환 

받는  상품이다.  이  상품이  출시되기  전  상식적인  모기지는  먼저  은행으로부터  주택  매입  자금을  빌려 

주택을  구입하고  빌린  돈은  추후  장기간에  걸쳐서  갚아  나갔던  것이다.

-  32  -

                                                                                        제54회  KBS  한국어능력시험(홀수형)

    일본  에도  시대  판화가인  우타가와  쿠니요시는  구부리고  있는  신체  모양을  활용해  얼굴을  그렸다. 

1847년에  완성한  작품  <얼핏  보면  무섭지만  실은  좋은  사람이다>를  보자.  다양한  자세를  취하고  있

는  여러  사람들을  합성하고  융합하여  인물의  얼굴을  표현했다.  이  작품을  통해  쿠니요시가  재정의한 

그림은  바로              ㉠              이다.  몸  전체보다는  얼굴이  작은  것이  당연하니  말이다.  그래서  이  작

품은  배보다  배꼽이  더  큰  그림이다.

    우리가  사용하는  물건  중에도  ㉡ 배보다  배꼽이  더  큰  제품들이  있다.  남자들이  보통  사용하는  면도

기가  그렇다.  처음  제품을  살  때는  본체와  면도날이  함께  묶여  있는  상품을  구입한다.  본체는  아주  오

래  사용할  수  있으나  본체에  탈부착하는  면도날은  일정  기간  사용하면  무뎌지게  되어  교환해야  한다. 

그런데  이  면도날  값이  비싸다.  게다가  본체와  면도날  간의  호환성이  적어  전에  구입했던  상품의  면

도날을  살  수  밖에  없다.

    이렇듯  예술과  경제는  관습과  상식에서  벗어난  창조적  융합을  통해  기존의  것들을  새롭게  정의한다

는  공통점이  있다.  이런  새로운  시도들이  침체되고  정체된  기존의  예술계와  경제계에  새로운  에너지

를  불어  넣음으로써  두  분야가  지속적으로  성장해  왔던  것이며,  앞으로도  그러할  것이다.
'''

In [28]:
cleaned_text = clean_text(text)
print(cleaned_text)

예술과 경제는 서로 다른 힘에 의해 움직이는 전혀 다른 세계일까? 그렇지 않다. 예술과 경제를 움 직이는 공통된 힘이 존재한다. 이 두 분야를 움직이는 힘이 같다면 각 분야가 작동하는 원리와 문제 를 해결하는 방법도 비슷할 것이다. 뛰어난 예술가들이 걸어 온 길을 살펴 우리의 경제와 기업 경영 에 활용해야 하는 이유가 바로 여기에 있다. 훌륭한 예술가는 이전의 상식을 허물고 새롭게 정의하는 재정의력을 활용해 독창적인 작품을 내놓 는다. “미술의 역사는 미술에 대한 재정의의 역사다.”라고 할 만큼 예술의 대가들은 나름대로 미술을 재정의하며 창작해 왔다. 관습적 결합과 익숙한 인과관계를 해체하고 새롭게 합성하는 창조적 융합을 통해 재정의력을 보여준 화가가 바로 초현실주의 화가 르네 마그리트다. 그의 <공동 발명>이란 작품 을 보자. 우리가 그동안 익숙하게 알고 있던 상식적인 인어와는 반대로 상반신이 어류이고 하반신이 사람이다. 이 그림을 통해 마그리트는 ‘거꾸로 뒤바꾸기’를 활용한 새로운 융합을 통해 인어를 재정의 했다. 너무나 익숙한 현실을 거꾸로 뒤집어 지극히 사실적으로 표현하는 것이 초현실주의 화가 마그 리트가 재정의한 미술이었다. 공동 발명 얼핏 보면 무섭지만 실은 좋은 사람이다 금융에서도 신상품을 개발하는데 ‘거꾸로 뒤바꾸기’를 통한 재정의력이 활용된 사례가 있다. 대표적 상품이 바로 역모기지대출이다. 역모기지대출이란 고령자가 자신의 집을 담보로 노후 생활 자금을 연 금 형태로 지급받고, 그 고령자가 사망한 후 은행이 주택을 처분하여 고령자에게 지급한 원금을 상환 받는 상품이다. 이 상품이 출시되기 전 상식적인 모기지는 먼저 은행으로부터 주택 매입 자금을 빌려 주택을 구입하고 빌린 돈은 추후 장기간에 걸쳐서 갚아 나갔던 것이다.


## 68회 시험지 PDFMiner

In [31]:
import json
import re

def clean_text(text):
    # 이중 띄어쓰기를 단일 띄어쓰기로 변경
    text = re.sub(r'\s+', ' ', text)
    # 불필요한 개행, 페이지 번호, 시험 정보 제거
    text = re.sub(r'-\s*\d+\s*-.*$', '', text, flags=re.MULTILINE)
    text = text.strip()
    return text

def convert_to_json(text):
    # 문제 구분 패턴
    problems = re.split(r'\n(\d+\.)', text)[1:]
    
    json_data = []
    
    for i in range(0, len(problems), 2):
        problem_number = problems[i].rstrip('.')
        problem_content = problems[i+1].strip()
        
        # 질문 추출
        question_match = re.search(r'(.*?)(\n[①②③④⑤].*)', problem_content, re.DOTALL)
        if question_match:
            question = clean_text(question_match.group(1))
            choices_text = question_match.group(2)
        else:
            question = ''
            choices_text = problem_content
        
        # 보기 추출
        choices = re.findall(r'[①②③④⑤]\s*(.*?)(?=[①②③④⑤]|$)', choices_text, re.DOTALL)
        choices = [clean_text(choice) for choice in choices]

        problem_entry = {
            "id": f"kbs-korean68-{problem_number}",
            "paragraph": "",
            "question": question,
            "choices": str(choices),
            "answer": "",
            "question_plus": ""
        }
        
        json_data.append(problem_entry)
    
    return json_data

In [32]:
with open('kbs_korean_68th.txt', 'r', encoding='utf-8') as file:
    text = file.read()

result = convert_to_json(text)

with open('temp.json', 'w', encoding='utf-8') as file:
    json.dump(result, file, ensure_ascii=False, indent=4)

In [33]:
import re

def clean_text(text):
    # 페이지 번호와 시험 정보 제거
    text = re.sub(r'-\s*\d+\s*-\n.*?KBS.*?시험\(.*?\)', '', text, flags=re.DOTALL)
    
    # 이중 이상 띄어쓰기를 단일 띄어쓰기로 변경
    text = re.sub(r'\s+', ' ', text)
    
    # 엔터로 구분된 문단을 줄글로 변경
    text = re.sub(r'\n+', ' ', text)
    
    return text.strip()

In [34]:
text = '''
무너지는  ㉠꽃  이파리처럼

휘날려  발  아래  깔리는

㉡서른  나문  해야

구름같이  피려던  뜻은  날로  굳어

한  금  두  금  곱다랗게  감기는  ㉢연륜(年輪)

갈매기처럼  꼬리  떨며

산호  핀  바다  바다에  나려앉은  ㉣섬으로  가자

비취빛  하늘  아래  피는  꽃은  맑기도  하리라

무너질  적에는  눈빛  파도에  적시우리

초라한  경력을  ㉤육지에  막은  다음

주름  잡히는  연륜마저  끊어버리고

나도  또한  불꽃처럼  열렬히  살리라

-  김기림,  「연륜」
'''

In [35]:
clean_text(text)

'무너지는 ㉠꽃 이파리처럼 휘날려 발 아래 깔리는 ㉡서른 나문 해야 구름같이 피려던 뜻은 날로 굳어 한 금 두 금 곱다랗게 감기는 ㉢연륜(年輪) 갈매기처럼 꼬리 떨며 산호 핀 바다 바다에 나려앉은 ㉣섬으로 가자 비취빛 하늘 아래 피는 꽃은 맑기도 하리라 무너질 적에는 눈빛 파도에 적시우리 초라한 경력을 ㉤육지에 막은 다음 주름 잡히는 연륜마저 끊어버리고 나도 또한 불꽃처럼 열렬히 살리라 - 김기림, 「연륜」'

In [36]:
text = '''
한산  통제영에서  장계를  쓰던  임진년의  여름밤은  달이  밝았다.  나는  내  무인된  운명을  깊

이  시름하였다.  한  자루의  칼과  더불어  나는  포위되어  있었고  세상의  덫에  걸려  있었지만, 

이  세상의  칼로  이  세상의  보이지  않는  덫을  칠  수는  없었다.  한산  통제영에서  그리고  그 

후의  여러  포구와  수영에서  나는  자주  식은땀을  흘렸고,  때때로  가엾고  안쓰러워서  칼을  버

리고  싶었다.

명량  전투에  관한  소문은  내가  보낸  장계의  범위를  넘는  것이었다.  그  소문은  명나라 

총병부의  정탐들이  퍼뜨리는  것  같았다.  나는  등골이  으스스했다.

명량의  장계를  보낸  지  두  달  만에  논공행상이  내려왔다.  선전관은  오지  않고,  조정의 

명을  받들어  도원수부가  시행했다.  거제  현령  안위가  정삼품  통정대부의  품계를  받았고 

전투에  참가했던  여러  읍진  수령과  군관들이  승진했다.  나에게는  상금으로  은전  스무  냥

을  보내왔다.  스무  냥의  무게와  질감은  섬뜩했다.  그  스무  냥  속에서  남쪽  바다를  들여
다보는  임금의  눈은  가늘게  번뜩이고  있었다.

[A]

스무  냥이  내려온  지  이틀  뒤에,  임금이  보낸  선전관  이원길이  목포  앞바다  고하도  수령

에  도착했다.  이원길은  수하를  거느리고  병영  막사  공사장까지  나를  찾아왔다.  서울  출신 

문관인데,  바다를  평생  처음  본다고  했다.  몸매가  가냘팠고  흰  손가락이  길었다.  먼  길을  온 

사람  같지  않게  그의  의관이  반듯했고  여독의  기색이  없었다.  수군  병영의  온갖  너저분한 

풍경에  그는  자주  눈살을  찌푸렸다.  나는  공사장  천막에서  그를  맞았다.  나는  인사했다.

―객고가  크시겠소.  전하께서  수군을  이처럼  염려하여  주시니  감읍할  뿐이오.

―전하의  근심이  실로  깊소이다.  달아난  배설  말이오.

명량  전투  직전에  탈영  도주한  경상  우수사  배설을  체포해서  끌고  가는  것이  임무라고  그

는  밝혔다.  그가  데리고  온  부하들  중에는  무관들이  섞여  있었다.  배설은  이미  수군에서  도

망쳤는데,  배설을  체포하는  일로  선전관이  남해의  수군  수영까지  온다는  것은  이해할  수  없

는  일이었다.

―배설은  이미  달아났지  않소?  배설을  잡으려면  이리로  오실  게  아니라  그의  본가  마을로 

가셔야  하지  않겠소?  경상도  성주  말이오.

―통제공,  그게  그리  간단치가  않소이다.  성주에도  군사들을  보냈으나  잡지  못했소.  배설이 

성주에  들어온  흔적도  찾지  못했소.  배설이  비록  달아났다  하나  본래  담력  있는  무장이었

소.  따르는  장졸들도  많았던  것으로  아오.  이자가  달아나서  대체  무슨  짓을  하려는  것인지, 

전하의  근심이  실로  여기에  있는  것이오.

나는  겨우  알았다.  임금이  수군통제사를  의심하고  있는  것이다.

명량  싸움의  결과가  임금은  두려운  것이다.  수영  안에  혹시라도  배설을  감추어놓고  역모

의  군사라도  기르고  있는  것이  아닌지,  그것이  임금의  조바심이었다.

이원길은  열흘  동안  수영에  머물렀다.  이원길은  데리고  온  수하들을  풀어  병영  안을  모두 

뒤졌고  수영  인근  백성들의  마을  헛간까지  뒤졌다.  이원길은  명량  전투  이전과  이후의  장졸

들의  숫자를  점검했고  각  읍진의  탈영자  숫자를  확인했다.  이원길의  수하들이  수영의  모든 
군관들을  불러서  배설의  탈영  경위와  탈영  직전  상황을  수사했다.  이원길의  수사의  초점은 

배설이  수영에서  탈영했느냐  아니냐에  맞추어져  있었다.  이원길은  귀로에  우수영,  벽파진, 

삼지원까지  뒤지고  돌아갔다.

나는  돌아가는  이원길을  전송하지  않았다.  이원길이  돌아가는  날짜를  나는  알지  못했다. 

그날  나는  목수들을  데리고  앞섬의  산속으로  들어가  신축  막사에  쓸  재목을  실어내고  있었

다.  산속  가파른  비탈에서  목수  한  명이  굴러내리는  나무에  깔려  죽었다.  내  종사관  김수철
이  돌아가는  이원길  일행에게  점심을  차려내고  건어물을  싸주어  보냈다.

-  18  -

                                                                                          제68회KBS한국어능력시험(홀수형)

이원길이  돌아간  지  보름  뒤에  임금이  보낸  ㉠  면사첩(免死帖)을  받았다.  도원수부의  행정

관이  면사첩을  들고  왔다.  ‘면사’  두  글자뿐이었다.  다른  아무  문구도  없었다.  조정을  능멸하

고  임금을  기만했으며  임금의  기동출격  명령에  따르지  않은  죄에  대하여  죽음을  면해주겠다
는  것이었다.  면사첩을  받던  날은  하루  종일  비가  내렸다.  나는  ‘면사’  두  글자를  오랫동안 

들여다보았다.  죄가  없다는  것도  아니고  죄를  사면해  주겠다는  것도  아니고  다만  죽이지는 

않겠다는  것이었다.

너를  죽여  마땅하지만  죽이지는  않겠다고  임금은  멀리서  그렇게  말하고  있었다.  ‘면사’  두 

글자  속에서,  뒤척이며  돌아눕는  임금의  해소  기침  소리가  들리는  듯했다.  글자  밑의  옥새

는  인주가  묻어날  듯이  새빨갰다.  칼을  올려놓은  시렁  아래  면사첩을  걸었다.  저  칼이  나의 
칼인가  임금의  칼인가.  면사첩  위  시렁에서  내  환도  두  자루는  나를  베는  임금의  칼처럼  보

였다.

                                                                                                      -  김훈,  『칼의  노래』
'''

In [37]:
clean_text(text)

'한산 통제영에서 장계를 쓰던 임진년의 여름밤은 달이 밝았다. 나는 내 무인된 운명을 깊 이 시름하였다. 한 자루의 칼과 더불어 나는 포위되어 있었고 세상의 덫에 걸려 있었지만, 이 세상의 칼로 이 세상의 보이지 않는 덫을 칠 수는 없었다. 한산 통제영에서 그리고 그 후의 여러 포구와 수영에서 나는 자주 식은땀을 흘렸고, 때때로 가엾고 안쓰러워서 칼을 버 리고 싶었다. 명량 전투에 관한 소문은 내가 보낸 장계의 범위를 넘는 것이었다. 그 소문은 명나라 총병부의 정탐들이 퍼뜨리는 것 같았다. 나는 등골이 으스스했다. 명량의 장계를 보낸 지 두 달 만에 논공행상이 내려왔다. 선전관은 오지 않고, 조정의 명을 받들어 도원수부가 시행했다. 거제 현령 안위가 정삼품 통정대부의 품계를 받았고 전투에 참가했던 여러 읍진 수령과 군관들이 승진했다. 나에게는 상금으로 은전 스무 냥 을 보내왔다. 스무 냥의 무게와 질감은 섬뜩했다. 그 스무 냥 속에서 남쪽 바다를 들여 다보는 임금의 눈은 가늘게 번뜩이고 있었다. [A] 스무 냥이 내려온 지 이틀 뒤에, 임금이 보낸 선전관 이원길이 목포 앞바다 고하도 수령 에 도착했다. 이원길은 수하를 거느리고 병영 막사 공사장까지 나를 찾아왔다. 서울 출신 문관인데, 바다를 평생 처음 본다고 했다. 몸매가 가냘팠고 흰 손가락이 길었다. 먼 길을 온 사람 같지 않게 그의 의관이 반듯했고 여독의 기색이 없었다. 수군 병영의 온갖 너저분한 풍경에 그는 자주 눈살을 찌푸렸다. 나는 공사장 천막에서 그를 맞았다. 나는 인사했다. ―객고가 크시겠소. 전하께서 수군을 이처럼 염려하여 주시니 감읍할 뿐이오. ―전하의 근심이 실로 깊소이다. 달아난 배설 말이오. 명량 전투 직전에 탈영 도주한 경상 우수사 배설을 체포해서 끌고 가는 것이 임무라고 그 는 밝혔다. 그가 데리고 온 부하들 중에는 무관들이 섞여 있었다. 배설은 이미 수군에서 도 망쳤는데, 배설을 체포하는 일로 선전관이 남해의 수군 수영까지 온다는 것은 이해할 수 없 는 일이었다. ―배설은 이미 달아

In [38]:
text = '''
성직자들은  성서에도  사람을  죽인  동물은  유죄이며  불결하기  때문에  죽여야  한다고  적혀 

있다고  강조했다. ｢출애굽기｣에는  “남자나  여자를  죽인  황소는  돌로  쳐  죽여야  하고  그  고

기를  먹어서는  안  된다.  그러나  소의  주인은  죄가  없다”라는  구절이  있다.  그래서  중세에는 

적지  않은  기독교  작가들이  동물한테도  스스로  저지른  행동에  대한  책임이  어느  정도는  있

다고  생각했다.  모든  살아  있는  존재가  그러하듯이  동물도  영혼을  지니는데,  생명의  숨결인 

그것은  죽은  뒤  신에게  돌아간다.  영혼은  생육을  하며,  식물의  경우와  마찬가지로  그것은 

영양·성장·재생의  근원이다.  영혼은  감지를  하며,  그것은  모든  감각의  근원이자  인간과  유사

한  정도로  고등하다고  판단되는  동물들이  지닌  지성의  근원이기도  하다.  그래서  많은  작가

들이  동물도  꿈을  꾸고  식별하며  추론하고  기억하며,  새로운  습성을  지닐  수  있다고  보았

다.

그러나  동물이  인간처럼  사고의  원천과  영적인  원천을  지니고  있는지가  문제시되었다.  토

마스  아퀴나스는  이  두  성질은  인간만이  지닌  고유한  것이라고  밝혔다.  고등한  동물은  분

명히  감각적인  인식  능력과  어떤  종류의  실천적  지성을  갖추고  있고,  어떤  감정  상태에  이

를  수도  있다.  하지만  비물질적인  것을  지각할  수는  없다.  자신에게  친숙한  ‘집’을  분간할 

수는  있지만,  ‘집’이라는  추상  관념에  이를  수는  없다는  것이다.  알베르투스  마그누스는  동

물이  어떻게  추론  능력을  지니고  있는지를  보여  주면서  다른  능력의  한계를  제시했다.  그

는  지능이  높은  동물에게도  신호는  그  자체로  머물러  있을  뿐,  결코  오늘날  상징이라고  부

르는  것이  되지는  않는다고  강조했다.

이  두  개의  근본적인  차이가  인간과  동물  사이에  뛰어넘을  수  없는  벽을  세운다고  볼  수 

있다.  동물은  실체가  없는  것을  지각하지  못한다.  모든  종교적·도덕적  관념과  추상적  관념

은  동물에게  금지된다.  그래서  토마스  아퀴나스는  동물을  재판에  부치는  것에  반대했다.  동

물은  사물의  숫자,  나아가  신호까지도  인식할  수  있지만  선악을  구분하지는  못한다는  이유

에서였다.  그렇지만  스콜라  신학은  동물의  내세와  현세에  관해  수많은  물음을  던지고  동물

을  도덕적인  책임이  있는  존재로  다룰  수  있을지를  계속해서  물었다.  그리고  토마스  아퀴

나스의  권위에도  불구하고  중세  말의  신학자들과  법학자들은  대부분  그  물음에  계속  긍정

적으로  답했다.

하지만  17세기가  되면서  상황이  변화했다.                  ㉠                철학자들이  등장했다.  예

컨대  데카르트에게  동물은  영혼을  가지고  있지  않으며  이성적  능력도  없는  존재였다.  동물

은  오롯이  ㉡  기계론적  기관이었다.  얼마  뒤  라메트라는  이러한  이론을  인간에게까지  넓혀 

적용하였다.  말브랑슈에  따르면,  동물은  번민을  알지  못한다.  번민은  원죄가  가져온  것이고, 

동물은  원죄와는  관련이  없기  때문이다.  동물을  도덕적이고,  책임이라는  관념을  지니고,  선

으로  향하는  것이  가능한  존재라고  생각하는  것에  대해  점점  더  많은  작가들이  터무니없다

고  여기게  되었다.  라신은 소송광이라는  희곡  작품에서,  당뎅  판사가  식용  닭을  훔친  개

에게  갤리선으로  보내  노를  젓게  하는  형벌을  내린  판결을  우스갯거리로  다루었다.
'''

In [39]:
clean_text(text)

'성직자들은 성서에도 사람을 죽인 동물은 유죄이며 불결하기 때문에 죽여야 한다고 적혀 있다고 강조했다. ｢출애굽기｣에는 “남자나 여자를 죽인 황소는 돌로 쳐 죽여야 하고 그 고 기를 먹어서는 안 된다. 그러나 소의 주인은 죄가 없다”라는 구절이 있다. 그래서 중세에는 적지 않은 기독교 작가들이 동물한테도 스스로 저지른 행동에 대한 책임이 어느 정도는 있 다고 생각했다. 모든 살아 있는 존재가 그러하듯이 동물도 영혼을 지니는데, 생명의 숨결인 그것은 죽은 뒤 신에게 돌아간다. 영혼은 생육을 하며, 식물의 경우와 마찬가지로 그것은 영양·성장·재생의 근원이다. 영혼은 감지를 하며, 그것은 모든 감각의 근원이자 인간과 유사 한 정도로 고등하다고 판단되는 동물들이 지닌 지성의 근원이기도 하다. 그래서 많은 작가 들이 동물도 꿈을 꾸고 식별하며 추론하고 기억하며, 새로운 습성을 지닐 수 있다고 보았 다. 그러나 동물이 인간처럼 사고의 원천과 영적인 원천을 지니고 있는지가 문제시되었다. 토 마스 아퀴나스는 이 두 성질은 인간만이 지닌 고유한 것이라고 밝혔다. 고등한 동물은 분 명히 감각적인 인식 능력과 어떤 종류의 실천적 지성을 갖추고 있고, 어떤 감정 상태에 이 를 수도 있다. 하지만 비물질적인 것을 지각할 수는 없다. 자신에게 친숙한 ‘집’을 분간할 수는 있지만, ‘집’이라는 추상 관념에 이를 수는 없다는 것이다. 알베르투스 마그누스는 동 물이 어떻게 추론 능력을 지니고 있는지를 보여 주면서 다른 능력의 한계를 제시했다. 그 는 지능이 높은 동물에게도 신호는 그 자체로 머물러 있을 뿐, 결코 오늘날 상징이라고 부 르는 것이 되지는 않는다고 강조했다. 이 두 개의 근본적인 차이가 인간과 동물 사이에 뛰어넘을 수 없는 벽을 세운다고 볼 수 있다. 동물은 실체가 없는 것을 지각하지 못한다. 모든 종교적·도덕적 관념과 추상적 관념 은 동물에게 금지된다. 그래서 토마스 아퀴나스는 동물을 재판에 부치는 것에 반대했다. 동 물은 사물의 숫자, 나아가 신호까지도 인식할 수 있지만 선

In [40]:
text = '''
지식은  어떤  사건  또는  사물의  인과  관계에  대한  기억에서  출발한다.  사물의  특성이나  사

건의  흐름을  관찰하고  경험한  것은  하나의  기억으로  남는다.  이러한  경험들이  남긴  기억에

서는  어떤  사물들에  공통으로  나타나는  특질들,  어떤  사건의  전후  관계로  일어나는  공통적 

요소들을  발견하는  추상화  과정이  진행되고,  그러면서  지식은  규칙이나  법칙으로  구체화된

다.  지능은  알고  있던  지식을  적용하는  방법이다.  그리고  지능을  일으키는  원천은  바로  지

식이다.  지식을  활용하는  추론,  추리  과정을  통해  지능은  관찰하거나  경험하지  못한  것에 

대한  인과를  끌어낼  수  있다.  그러므로  지능의  의미는  지식의  추론을  통한  새로운  지식의 

창출  활동이라고  할  만하다.  따라서  지능  활동은  추론이나  추리  과정이라  볼  수  있다.

전통적으로  정보를  지식으로  바꾸는  과정은  사람만이  할  수  있었다.  인간은  지식을  담아

내는  수단도  함께  만들었는데,  우리가  쓰는  말,  문자,  그림  따위가  지식을  기록하고  저장하

기  위한  대표적인  것이다.  인간의  사상  또는  감정을  표현한  창작물은  저작물이라  하여  법에

서  저작권의  대상으로  보호한다.  현대에  들어서는  디지털  정보  기술이  급격히  발달하면서 

컴퓨터가  우리의  일상생활에  꼭  필요한  필수  요소로  자리  잡았다.  창작물을  저장하고  이용

하는  수단  또한  디지털  정보  기술을  활용하게  되었다.  가상  현실,  게임과  같은  새로운  형식

도  출현하고  있다.  이들은  글이나,  말,  음악,  영상의  요소가  디지털  데이터로  변환된  것으로 

본질적으로  전통적  표현  형식을  크게  벗어났다고까지  할  것은  아니다.  다만  사람의  생각이

나  의도를  담을  수  있도록  구성된  장치들이  활동하면서  여러  데이터를  생성할  수  있다는  것

이고,  그리하여  이들도  사람의  창작물로  인정될  여지가  있다는  것이다.

어떤  제품을  생산하는  과정에  대해서는  일반적으로  사람의  생각이나  의도를  담아내는  공

정은  아니라고들  여긴다.  사람은  제품을  생산하기  위해  여러  기기를  모아서  제조  과정을  구

성하고,  정해진  순서에  따라  그  기기들을  조작한다.  이  과정에서  특정한  조절  요소를  조화

롭게  설정할  경우,  그로  말미암아  품질의  개선이나  생산량의  확대가  뒤따른다.  그것을  우리

는  ‘노하우’라고  부른다.  이것을  창작의  산물이  아니라  보기도  하지만,  노하우는  생산  현장

의  지식이며  그  가치  또한  인정되는  것이  사실이다.  만일  노하우와  같은  생산  현장의  지식

을  전통적인  형식,  즉  언어적  저작물,  촬영  영상,  그래프  등으로  표현하게  된다면,  이  또한 

-  21  -

                                                                                          제68회KBS한국어능력시험(홀수형)

창작물의  범주에  넣어  관리할  수  있는  여지가  있다.

특히  디지털  정보  기술을  적용한  생산  과정에서는,  이  과정에  따라  발생한  데이터를  모두 

모아  콘텐츠의  형식으로  표현할  수  있다.  이런  노하우의  가치가  담긴  콘텐츠  데이터를  어떤 

규칙이나  알고리즘으로  처리하면,  마찬가지로  새로운  현장에  ‘재현’하거나  ‘적용’할  수  있는 

지식이  된다.  이는  새로운  형태의  저작물이  될  수  있지  않을까?  이들은  전통적인  형식의  표

현  수단이  아니라  ‘데이터’라는  새로운  기호  매체를  이용하고  있으며,  담고  있는  내용도  직

접  드러나지  않고  어떤  처리  과정을  거쳐야만  발현되도록  내재되어  있는  형태를  가진  산물

이다.  오늘날  지식  환경의  변화는  이들도  저작물의  범주에  넣는  과제를  고려하도록  하는  상

황이라  하겠다.'''

In [41]:
clean_text(text)

'지식은 어떤 사건 또는 사물의 인과 관계에 대한 기억에서 출발한다. 사물의 특성이나 사 건의 흐름을 관찰하고 경험한 것은 하나의 기억으로 남는다. 이러한 경험들이 남긴 기억에 서는 어떤 사물들에 공통으로 나타나는 특질들, 어떤 사건의 전후 관계로 일어나는 공통적 요소들을 발견하는 추상화 과정이 진행되고, 그러면서 지식은 규칙이나 법칙으로 구체화된 다. 지능은 알고 있던 지식을 적용하는 방법이다. 그리고 지능을 일으키는 원천은 바로 지 식이다. 지식을 활용하는 추론, 추리 과정을 통해 지능은 관찰하거나 경험하지 못한 것에 대한 인과를 끌어낼 수 있다. 그러므로 지능의 의미는 지식의 추론을 통한 새로운 지식의 창출 활동이라고 할 만하다. 따라서 지능 활동은 추론이나 추리 과정이라 볼 수 있다. 전통적으로 정보를 지식으로 바꾸는 과정은 사람만이 할 수 있었다. 인간은 지식을 담아 내는 수단도 함께 만들었는데, 우리가 쓰는 말, 문자, 그림 따위가 지식을 기록하고 저장하 기 위한 대표적인 것이다. 인간의 사상 또는 감정을 표현한 창작물은 저작물이라 하여 법에 서 저작권의 대상으로 보호한다. 현대에 들어서는 디지털 정보 기술이 급격히 발달하면서 컴퓨터가 우리의 일상생활에 꼭 필요한 필수 요소로 자리 잡았다. 창작물을 저장하고 이용 하는 수단 또한 디지털 정보 기술을 활용하게 되었다. 가상 현실, 게임과 같은 새로운 형식 도 출현하고 있다. 이들은 글이나, 말, 음악, 영상의 요소가 디지털 데이터로 변환된 것으로 본질적으로 전통적 표현 형식을 크게 벗어났다고까지 할 것은 아니다. 다만 사람의 생각이 나 의도를 담을 수 있도록 구성된 장치들이 활동하면서 여러 데이터를 생성할 수 있다는 것 이고, 그리하여 이들도 사람의 창작물로 인정될 여지가 있다는 것이다. 어떤 제품을 생산하는 과정에 대해서는 일반적으로 사람의 생각이나 의도를 담아내는 공 정은 아니라고들 여긴다. 사람은 제품을 생산하기 위해 여러 기기를 모아서 제조 과정을 구 성하고, 정해진 순서에 따라 그 기기들을 조

In [42]:
text = '''
양자  역학에서  EPR  역설은  물리량의  측정  문제를  제기한  사고  실험이다.  사고  실험이란 

머릿속에서  생각으로  진행하는  실험으로  실험에  필요한  장치와  조건을  가정한  후  이론을  바

탕으로  일어날  결과를  예측한다.  1935년  ⓐ 아인슈타인(Einstein)은  포돌스키(Podolsky),  로

젠(Rogen)과  함께  양자  역학이  논리적으로  완전하지  않음을  보이기  위해  이  역설을  발표했

으며,  EPR은  그들의  이름을  딴  것이다.  이  역설은  양자  역학의  주류인  코펜하겐학파의  해

석에  대한  문제  제기에서  시작한다.  코펜하겐학파의  해석에  의하면  전자(電子)는  파동함수

로  표현할  수  있는데,  여러  가지  상태가  확률적으로  겹쳐있으며,  관측을  하면  그와  동시에 

파동함수가  붕괴되어  더  이상  겹침  상태가  아니라  단  하나의  상태로  결정된다.  EPR  역설은 

코펜하겐학파  해석에서  여러  상태가  확률적으로  겹쳐있다는  양자  중첩의  개념을  반박하고자 

제시되었다.  이를  이해하기  위해서는  국소성  개념의  이해가  필요하다. 

물리학에서는  국소성  이론이  통용되는데,  국소성이란  어떤  물체가  주변에  주는  물리적  영

향은  바로  인접한  영역으로만  전파된다는  성질을  말하며,  공간을  뛰어넘어  순간적으로  영향

을  미칠  수  없다는  뜻이다.  이는  물리적  영향을  매개하는  무엇인가가  존재하여  그것을  통하

여  그  영향이  차례차례  전달된다는  것이다.  아인슈타인의  상대성  이론에  따르면,  매개물을 

통한  영향의  전달에는  유한한  시간이  걸리며  그  속도는  진공  중의  빛의  속도를  넘을  수  없

다.  따라서  떨어진  두  물체  사이에서  국소성이란  빛보다  빠른  영향의  전달이  불가능함을  의

미한다. 

EPR  역설을  이해하기  위해  검은색  공과  하얀색  공이  들어있는  상자가  각각  있다고  가정

해보자.  어떤  색의  공이  들어있는  상자인지  모른  채  두  상자를  수십  광년  떨어뜨려  놓은  후 

한쪽  상자에  들어있는  공의  색을  관찰하면  다른  상자에  들어있는  공의  색을  알  수가  있다. 

즉  한쪽  상자를  열어  색을  관찰하면  다른  쪽  상자를  열어서  관측하지  않고도  그  색을  알  수 

있다는  것이다.  그런데  이  두  가지  색의  공들은  서로  국소성의  원리가  작용하지  않을  만큼 

먼  거리에  있으며  빛보다  빠른  존재가  있을  수  없음에도  불구하고  다른  한쪽의  상태를  알 

수  있는,  달리  말해  광속보다  빠른  원격  작용이  일어나는  셈이라는  것이다.  아인슈타인은 

이러한  EPR  역설을  통해  양자  역학이  예측하는  바가  ㉠  유령  같은  원격  작용이라고  지칭하

며  양자  중첩의  개념을  부정하고자  했다. 

양자  역학은  확률적이다.  우리는  무엇인가  정확하게  알지  못하는  것이  있을  때  확률로  표

현한다.  하이젠베르크는  불확정성  원리로  그  이유를  설명했다.  운동량과  위치를  동시에  정

확히  측정할  수  없기  때문에  확률의  개념이  필요하다는  것이다.  하지만  아인슈타인이  주도

한  EPR  역설은  여기에  의문을  제기한다.  아인슈타인은  우리는  원칙적으로  자연에  대해  완

벽하게  알  수  있으나  단지  지금  우리가  알지  못하는  ‘무언가’가  있는데,  그것  때문에  양자 

-  23  -

                                                                                          제68회KBS한국어능력시험(홀수형)

역학에서  확률의  개념인  양자  중첩이  나오는  것이라고  보았다.  그  알지  못하는  ‘무언가’를 

알게  되었을  때,  양자  역학에서  확률은  제거된다는  것이다.  그  ‘무언가’를  숨은  변수라  하며, 

원자  수준의  실제  실험을  통해  오늘날  대부분  과학자는  숨은  변수는  없고,  아인슈타인이  틀

렸다고  생각한다. '''

In [43]:
clean_text(text)

'양자 역학에서 EPR 역설은 물리량의 측정 문제를 제기한 사고 실험이다. 사고 실험이란 머릿속에서 생각으로 진행하는 실험으로 실험에 필요한 장치와 조건을 가정한 후 이론을 바 탕으로 일어날 결과를 예측한다. 1935년 ⓐ 아인슈타인(Einstein)은 포돌스키(Podolsky), 로 젠(Rogen)과 함께 양자 역학이 논리적으로 완전하지 않음을 보이기 위해 이 역설을 발표했 으며, EPR은 그들의 이름을 딴 것이다. 이 역설은 양자 역학의 주류인 코펜하겐학파의 해 석에 대한 문제 제기에서 시작한다. 코펜하겐학파의 해석에 의하면 전자(電子)는 파동함수 로 표현할 수 있는데, 여러 가지 상태가 확률적으로 겹쳐있으며, 관측을 하면 그와 동시에 파동함수가 붕괴되어 더 이상 겹침 상태가 아니라 단 하나의 상태로 결정된다. EPR 역설은 코펜하겐학파 해석에서 여러 상태가 확률적으로 겹쳐있다는 양자 중첩의 개념을 반박하고자 제시되었다. 이를 이해하기 위해서는 국소성 개념의 이해가 필요하다. 물리학에서는 국소성 이론이 통용되는데, 국소성이란 어떤 물체가 주변에 주는 물리적 영 향은 바로 인접한 영역으로만 전파된다는 성질을 말하며, 공간을 뛰어넘어 순간적으로 영향 을 미칠 수 없다는 뜻이다. 이는 물리적 영향을 매개하는 무엇인가가 존재하여 그것을 통하 여 그 영향이 차례차례 전달된다는 것이다. 아인슈타인의 상대성 이론에 따르면, 매개물을 통한 영향의 전달에는 유한한 시간이 걸리며 그 속도는 진공 중의 빛의 속도를 넘을 수 없 다. 따라서 떨어진 두 물체 사이에서 국소성이란 빛보다 빠른 영향의 전달이 불가능함을 의 미한다. EPR 역설을 이해하기 위해 검은색 공과 하얀색 공이 들어있는 상자가 각각 있다고 가정 해보자. 어떤 색의 공이 들어있는 상자인지 모른 채 두 상자를 수십 광년 떨어뜨려 놓은 후 한쪽 상자에 들어있는 공의 색을 관찰하면 다른 상자에 들어있는 공의 색을 알 수가 있다. 즉 한쪽 상자를 열어 색을 관찰하면 다른 쪽 상자를 열어서 관측하지 않고도 그 색을 알 수

In [44]:
text = '''
힘을  받은  용수철은  일정  길이만큼  순간적으로  순식간에  늘어난다.  재료에  가해지는  힘의 

방향과  수직인  면의  단위  면적당  받는  힘을  응력이라  하는데,  추를  제거하여  응력이  없어지

면  용수철은  곧바로  원래의  길이로  돌아간다.  이처럼  가한  힘에  상응하여  순간적으로  변형

이  일어나고  힘을  제거하면  원래의  위치로  돌아가는  성질을  탄성이라  한다.  탄성을  갖는  물

체도  탄성  한계를  넘는  변형을  시키면  가해준  힘을  제거해도  원래의  상태로  돌아가지  않는 

영구적인  변형이  생긴다.  탄성  한계에서는  변형과  응력은  정비례의  관계를  갖는다.  이에  반

해  물이나  물렁물렁한  엿과  같은  물질은  힘을  가하면  시간에  따라  물체가  흐르며  변형이 

생기고,  응력을  제거하여도  원래의  위치로  돌아가지  않는데  이러한  성질을  점성이라  한다. 

액체와  같이  점성을  갖는  물체의  변형은  탄성을  갖는  물체와  달리  즉각적이지  않고  영구적

이다.

탄성과  점성을  동시에  갖는  물체를  점탄성체라고  하는데  대부분의  고분자  물질들은  점탄

성체이다.  점탄성체에서  관찰할  수  있는  대표적인  현상이  응력  완화와  크리프이다.  고분자 

물질인  기타  줄의  음고(音高)는  기타  줄의  장력에  비례하여  커지는데,  기타  줄을  일정  길이 

만큼  늘려  음고를  고정한다.  이때  장력이란  일정  길이만큼  늘릴  때  가해준  힘에  대해  즉각

적으로  변형이  일어나는  탄성의  성질을  나타낸다.  그러나  변형이  고정된  후  시간이  많이  지

나면  기타  줄에  걸린  장력이  떨어져  음고가  낮아진다.  이처럼  탄성  한계  내에서  점탄성체의 

변형을  일정하게  유지했을  때  응력이  시간에  따라  줄어드는  현상을  응력  완화라고  한다.  응

력  완화가  일어나는  이유는  점탄성체를  이루는  분자들이  응력을  완화하는  상태로  천천히 

재배열하면서  응력이  줄어들기  때문이다. 

이와  달리  크리프는  점탄성체에  고정된  응력을  받게  했을  때  물체가  서서히  영구적인  변

형이  일어나는  것을  일컫는다.  플라스틱  재질의  책  선반에  일정한  무게의  책을  올려놓으면 

책  선반이  아래로  즉각적으로  휘게  되는데,  책을  선반에서  제거하면  원래의  위치로  바로  돌

아온다.  원위치로  선반이  바로  돌아온다는  것은  탄성  한계  이내의  탄성  변형이라는  것을  의

미한다.  그런데  같은  무게의  책을  선반에  올린  상태에서  오랜  시간  동안  관찰하면  선반이 

천천히  아래로  휘게  되고  ㉠  책을  제거하여도  선반은  원래의  위치로  돌아오지  않는다.  이처

럼  어떤  물체에  일정한  힘을  가해  주었을  때  시간이  지남에  따라  물체가  천천히  변형되는 

 

현상을  크리프라  한다.  크리프  역시  점탄성체를  구성하는  분자들이  응력에  의해  재배열하면

서  변형이  서서히  일어나는  것이다.  점탄성체가  갖는  탄성과  점성  중  어느  것이  더  많이  발

현되는가는  온도에도  크게  의존한다.  온도가  높을수록  점탄성체  분자의  유동이  원활해져서 

크리프와  응력  강화가  가속화된다. 
'''

In [45]:
clean_text(text)

'힘을 받은 용수철은 일정 길이만큼 순간적으로 순식간에 늘어난다. 재료에 가해지는 힘의 방향과 수직인 면의 단위 면적당 받는 힘을 응력이라 하는데, 추를 제거하여 응력이 없어지 면 용수철은 곧바로 원래의 길이로 돌아간다. 이처럼 가한 힘에 상응하여 순간적으로 변형 이 일어나고 힘을 제거하면 원래의 위치로 돌아가는 성질을 탄성이라 한다. 탄성을 갖는 물 체도 탄성 한계를 넘는 변형을 시키면 가해준 힘을 제거해도 원래의 상태로 돌아가지 않는 영구적인 변형이 생긴다. 탄성 한계에서는 변형과 응력은 정비례의 관계를 갖는다. 이에 반 해 물이나 물렁물렁한 엿과 같은 물질은 힘을 가하면 시간에 따라 물체가 흐르며 변형이 생기고, 응력을 제거하여도 원래의 위치로 돌아가지 않는데 이러한 성질을 점성이라 한다. 액체와 같이 점성을 갖는 물체의 변형은 탄성을 갖는 물체와 달리 즉각적이지 않고 영구적 이다. 탄성과 점성을 동시에 갖는 물체를 점탄성체라고 하는데 대부분의 고분자 물질들은 점탄 성체이다. 점탄성체에서 관찰할 수 있는 대표적인 현상이 응력 완화와 크리프이다. 고분자 물질인 기타 줄의 음고(音高)는 기타 줄의 장력에 비례하여 커지는데, 기타 줄을 일정 길이 만큼 늘려 음고를 고정한다. 이때 장력이란 일정 길이만큼 늘릴 때 가해준 힘에 대해 즉각 적으로 변형이 일어나는 탄성의 성질을 나타낸다. 그러나 변형이 고정된 후 시간이 많이 지 나면 기타 줄에 걸린 장력이 떨어져 음고가 낮아진다. 이처럼 탄성 한계 내에서 점탄성체의 변형을 일정하게 유지했을 때 응력이 시간에 따라 줄어드는 현상을 응력 완화라고 한다. 응 력 완화가 일어나는 이유는 점탄성체를 이루는 분자들이 응력을 완화하는 상태로 천천히 재배열하면서 응력이 줄어들기 때문이다. 이와 달리 크리프는 점탄성체에 고정된 응력을 받게 했을 때 물체가 서서히 영구적인 변 형이 일어나는 것을 일컫는다. 플라스틱 재질의 책 선반에 일정한 무게의 책을 올려놓으면 책 선반이 아래로 즉각적으로 휘게 되는데, 책을 선반에서 제거하면 원래의

In [46]:
text = '''
고장  난  전차  문제는  영국  철학자인  필리파  풋이  1967년에  처음  제시한  철학  문제이다. 

브레이크가  고장  난  전차가  가던  선로에는  5명의  사람이  있는데,  지선(支線)에는  1명의  사

람이  있다.  이때  당신이  전차의  기관사라면  가던  쪽으로  계속  갈  것인가,  아니면  지선으로 

방향을  틀  것인가? 

만약  전차가  고장  난  채로  계속  가도록  내버려  둔다면,  5명의  사람이  죽게  된다.  만약  방

향을  틀게  된다면  1명의  사람이  죽게  된다.  방향을  틀지  않으면  더  많은  사람이  죽게  되는

데,  살릴  수  있는  더  많은  사람을  죽게  했다는  비난을  받을  것이다.  반면에  방향을  틀게  되

면  더  적은  사람이  죽게  된다.  그러나  이것은  기관사가  그  사람을  일부러  죽인  것이다.  방

향을  틀지  않아  죽게  된  5명은  기관사가  의도적으로  죽인  것이  아니라  고장  난  기관차에 

의해  죽게  된  것이다.  기관사는  5명이  죽도록  내버려  둔  것뿐이다.  ‘죽이는  것’은  ‘죽게  내

버려  두는  것’보다  도덕적으로  나쁘다는  것이  사람들의  직관이다.  따라서  방향을  틀지  않아

도,  틀어도  비난을  받게  된다.

이  딜레마를  일반인에게  물어보면  ⓐ방향을  틀어야  한다고  생각하는  사람이  많다.  철학자 

중에서는  ‘이중  효과의  원리’로  방향을  틀어야  한다는  쪽을  옹호하는  사람들이  있다.  어떤 

행동에서  좋은  결과와  나쁜  결과가  모두  발생하는데  나쁜  결과를  직접  의도한  것이  아니라

면  그  행동은  허용  가능하다는  주장이다.  예컨대  남에게  고통을  주면  안  된다는  것은  기본

적인  윤리  원칙이지만  의료진이  주사를  놓는  행위를  비난하지  않는다.  주사를  놓은  행위는 

치료라는  좋은  결과를  위해  부수적으로  따르는  나쁜  결과이기  때문이다.  마찬가지로  고장 

난  전차의  방향을  튼  것은  5명을  살리겠다는  좋은  결과를  의도했지만  1명이  죽었다는  나쁜 

결과가  생겼다는  것이다. 

그러나  이런  답변에는  나쁜  결과가  생길지  뻔히  아는데도  의도하지  않았다고  말할  수  있

-  26  -

                                                                                          제68회KBS한국어능력시험(홀수형)

느냐는  비판이  제기된다.  ㉠  장기  탈취  사고  실험이  그것이다.  서로  다른  장기가  손상되어 

이식을  받지  않으면  곧  죽을  환자가  5명  있다고  하자.  한  명이  건강  검진을  받으러  왔는데 

이  사람은  5명에  적합한  장기를  모두  가지고  있다.  이  사람을  마취해  장기를  탈취하고서, 

그  사람이  죽은  것은  5명을  살린다는  좋은  결과를  위해  생긴  부수적인  나쁜  결과라고  주장

할  수  있을까?  1명이  죽는다는  것은  누구나  예측할  수  있는데  말이다.  이  사례는  고장  난 

전차  딜레마와  같은  상황이다.

방향을  틀어야  한다는  직관은  엄격하지  않다는  비판도  받는다.  위  사고  실험이  변형된  ㉡ 

‘육교  사고  실험’이  있다.  똑같이  전차의  브레이크가  고장  난  상황인데  전차가  가는  선로의 

육교  위에  한  사람이  서  있다.  이  사람은  덩치가  커서  선로로  떨어뜨리면  전차를  멈추게  할 

수  있고,  그러면  5명이  살게  된다고  하자.  이때  그를  밀어뜨려야  할까?  위  전차  사고  실험

에서는  방향을  틀어야  한다고  주장한  사람들은  이  실험에서는  밀어서는  안  된다는  의견이 

많았다.  ⓑ 육교  위의  사람을  미나  전차의  방향을  트나  5명을  살리기  위해  적극적인  행동을 

한  것은  똑같은데  말이다.  이것은  사람들의  직관이  그리  일관되지  못함을  보여  준다.
'''

In [47]:
clean_text(text)

'고장 난 전차 문제는 영국 철학자인 필리파 풋이 1967년에 처음 제시한 철학 문제이다. 브레이크가 고장 난 전차가 가던 선로에는 5명의 사람이 있는데, 지선(支線)에는 1명의 사 람이 있다. 이때 당신이 전차의 기관사라면 가던 쪽으로 계속 갈 것인가, 아니면 지선으로 방향을 틀 것인가? 만약 전차가 고장 난 채로 계속 가도록 내버려 둔다면, 5명의 사람이 죽게 된다. 만약 방 향을 틀게 된다면 1명의 사람이 죽게 된다. 방향을 틀지 않으면 더 많은 사람이 죽게 되는 데, 살릴 수 있는 더 많은 사람을 죽게 했다는 비난을 받을 것이다. 반면에 방향을 틀게 되 면 더 적은 사람이 죽게 된다. 그러나 이것은 기관사가 그 사람을 일부러 죽인 것이다. 방 향을 틀지 않아 죽게 된 5명은 기관사가 의도적으로 죽인 것이 아니라 고장 난 기관차에 의해 죽게 된 것이다. 기관사는 5명이 죽도록 내버려 둔 것뿐이다. ‘죽이는 것’은 ‘죽게 내 버려 두는 것’보다 도덕적으로 나쁘다는 것이 사람들의 직관이다. 따라서 방향을 틀지 않아 도, 틀어도 비난을 받게 된다. 이 딜레마를 일반인에게 물어보면 ⓐ방향을 틀어야 한다고 생각하는 사람이 많다. 철학자 중에서는 ‘이중 효과의 원리’로 방향을 틀어야 한다는 쪽을 옹호하는 사람들이 있다. 어떤 행동에서 좋은 결과와 나쁜 결과가 모두 발생하는데 나쁜 결과를 직접 의도한 것이 아니라 면 그 행동은 허용 가능하다는 주장이다. 예컨대 남에게 고통을 주면 안 된다는 것은 기본 적인 윤리 원칙이지만 의료진이 주사를 놓는 행위를 비난하지 않는다. 주사를 놓은 행위는 치료라는 좋은 결과를 위해 부수적으로 따르는 나쁜 결과이기 때문이다. 마찬가지로 고장 난 전차의 방향을 튼 것은 5명을 살리겠다는 좋은 결과를 의도했지만 1명이 죽었다는 나쁜 결과가 생겼다는 것이다. 그러나 이런 답변에는 나쁜 결과가 생길지 뻔히 아는데도 의도하지 않았다고 말할 수 있 느냐는 비판이 제기된다. ㉠ 장기 탈취 사고 실험이 그것이다. 서로 다른 장기가 손상되어 이식을 받지

In [48]:
text = '''
[삶과  문화]  ‘알아야  면장’…면장이  그  면장이  아니라고?

한국어는  유래나  어원이  밝혀지지  않은  말이  많다.  민간  어원도  뒤죽박죽  섞여서,  예컨대 

훈몽자회(1527)에  이미  ‘행주’가  나옴에도  ‘행주치마’가  행주대첩(1593)에서  유래한다고  잘

못  알려지기도  했다.  붓,  먹,  배추  등  언뜻  고유어  같지만  사실은  중국어  귀화어이거나  ‘김

치’처럼  한국  한자어가  변한  말도  있고  ‘생각’처럼  취음자  ‘生覺’  탓에  한자어  느낌이  드는 

말도  있다.

㉠ ‘알아야  면장’이  알고  보니  ‘面長’이  아니고  ‘面牆(담을  마주  대하듯  앞이  안  보임,  견문

이  좁음)을  피하다’의  ‘면면장(免面牆)’에서  왔다는  설은  2000년대  들어서부터  퍼졌다.  통념

을  뒤집으니  일단  왠지  그럴싸해  보이기도  한다.  그러나  다시  들여다보면  뭔가  석연치  않

다.  일상에서  잘  안  쓰는  말에  한자  한  자가  더  붙었다가  다시  빠져  속담이  됐다?  다들  ‘面

長’으로  알고  뜻도  잘  통하는데,  검증  없이  억지스레  갖다  붙인  느낌이다.  주로  ㉡  한문에서 

유래하는  고사성어와  달리  ㉢ 속담은  민간에서  전해지는  말이  대다수다.

‘면장(面牆)’은  중국어,  일본어  사전들에서  서경(書經),  후한서(後漢書)  등이  출처로  언급된

다.  그런데  한국  사이트나  기사,  책에서는  논어(論語)의  양화(陽貨)에  공자가  아들  백어에게 

공부를  강조하며  하는  말로  나오나,  정작  ‘면면장(免面墻)’이라는  구절은  없다.

人而不為  ‘周南’,  ‘召南’,  其猶正牆面而立也與?(사람이  ‘주남’과  ‘소남’을  배우지  않는다면 

담장을  마주하고  서  있는  것  같지  않겠는가?)

배우지  않으면  담에  가로막혀  앞도  안  보이고  한  발짝도  못  나가는  것이나  다름없다는  취

지는  어쨌든  같지만,  논어  구절은  ‘면장’도  아닌  ‘장면(牆面)’이며  ‘면할  면(免)’도  없고,  중국

어든  일본어든  ‘免面牆’이나  ‘免牆’이라는  숙어나  관용어도  안  쓴다.  한문을  좀  아는  호사가

들한테  논어부터  떠올라서  그랬겠지만  굳이  후한서나  서경을  출처로  언급하지  않고  ‘免面

牆’도  지어낸  데서  신빙성은  더  떨어진다.

‘알아야  면장’이  조선시대부터  쓰였다는  증거도  없다.  면장이라는  관직은  1910년  생겼다. 

이  속담은  1962년  민중서관에서  펴낸  ‘속담사전’에  나오고,  매일경제  1966년  12월  26일  자 

‘알아야  面長을  해’라는  제목의  기사를  볼  때,  아무리  일러야  1940~50년대쯤부터  퍼진  걸

로  짐작된다.  그때부터  퍼진  속담에  뜬금없이  논어든  다른  한문이든  영향을  미치긴  어렵다.

어원이  좀  알쏭달쏭하다고  여겼는지  표준국어대사전은  속담이  ‘면장(面長)’과  ‘면장(面牆)’ 

항목에는  없고  ‘알다’에  있다.  다만  ‘알아야  면장을  하지’의  뜻풀이가  ‘어떤  일이든  그  일을 

하려면  그것에  관련된  학식이나  실력을  갖추고  있어야  함을  비유적으로  이르는  말’이듯이 

‘면장(面長)’을  넌지시  드러낸다.

북한의  조선말대사전은  ‘면장’과  ‘알다’  항목에  ‘배워야  면장이라’와  ‘알아야  면장(도  하지)’

이  있고,  속담  풀이는  ‘그래도  얼마간의  지식이라도  있어야  면장  노릇을  한다는  뜻으로  아

-  30  -

                                                                                          제68회KBS한국어능력시험(홀수형)

무것이나  다  그  방면의  지식이  있어야  해당한  일을  맡아  할  수  있음을  비겨  이르는  말’이라 

나온다.

종합하면  ‘알아야  면장’이  ‘免面牆/免牆’이라는  설은  하나만  알고  둘은  몰라  생긴  가짜  어

원일  가능성이  높다.  통념이나  통설을  깨려면  제대로  깨야  한다.  통념을  깨는  재미에만  몰

두해  별다른  근거도  없이  꾸며내는  우를  범하면  안  된다.  꼭  통념을  깨야  한다는  것도  일종

의  통념이다.'''

In [49]:
clean_text(text)

'[삶과 문화] ‘알아야 면장’…면장이 그 면장이 아니라고? 한국어는 유래나 어원이 밝혀지지 않은 말이 많다. 민간 어원도 뒤죽박죽 섞여서, 예컨대 훈몽자회(1527)에 이미 ‘행주’가 나옴에도 ‘행주치마’가 행주대첩(1593)에서 유래한다고 잘 못 알려지기도 했다. 붓, 먹, 배추 등 언뜻 고유어 같지만 사실은 중국어 귀화어이거나 ‘김 치’처럼 한국 한자어가 변한 말도 있고 ‘생각’처럼 취음자 ‘生覺’ 탓에 한자어 느낌이 드는 말도 있다. ㉠ ‘알아야 면장’이 알고 보니 ‘面長’이 아니고 ‘面牆(담을 마주 대하듯 앞이 안 보임, 견문 이 좁음)을 피하다’의 ‘면면장(免面牆)’에서 왔다는 설은 2000년대 들어서부터 퍼졌다. 통념 을 뒤집으니 일단 왠지 그럴싸해 보이기도 한다. 그러나 다시 들여다보면 뭔가 석연치 않 다. 일상에서 잘 안 쓰는 말에 한자 한 자가 더 붙었다가 다시 빠져 속담이 됐다? 다들 ‘面 長’으로 알고 뜻도 잘 통하는데, 검증 없이 억지스레 갖다 붙인 느낌이다. 주로 ㉡ 한문에서 유래하는 고사성어와 달리 ㉢ 속담은 민간에서 전해지는 말이 대다수다. ‘면장(面牆)’은 중국어, 일본어 사전들에서 서경(書經), 후한서(後漢書) 등이 출처로 언급된 다. 그런데 한국 사이트나 기사, 책에서는 논어(論語)의 양화(陽貨)에 공자가 아들 백어에게 공부를 강조하며 하는 말로 나오나, 정작 ‘면면장(免面墻)’이라는 구절은 없다. 人而不為 ‘周南’, ‘召南’, 其猶正牆面而立也與?(사람이 ‘주남’과 ‘소남’을 배우지 않는다면 담장을 마주하고 서 있는 것 같지 않겠는가?) 배우지 않으면 담에 가로막혀 앞도 안 보이고 한 발짝도 못 나가는 것이나 다름없다는 취 지는 어쨌든 같지만, 논어 구절은 ‘면장’도 아닌 ‘장면(牆面)’이며 ‘면할 면(免)’도 없고, 중국 어든 일본어든 ‘免面牆’이나 ‘免牆’이라는 숙어나 관용어도 안 쓴다. 한문을 좀 아는 호사가 들한테 논어부터 떠올라서 그랬겠지만 굳이 후한서나 서경을 출처로 언급하지 않고 ‘免面 牆’도 지어낸 데서 신빙성은

## 'train_reformat + kbs_korean_data'  concat

In [53]:
import pandas as pd

kbs_df = pd.read_csv('kbs_korean_data.csv')
train_df = pd.read_csv('../../resources/raw/train_reformat.csv')

In [56]:
kbs_train = pd.concat([kbs_df, train_df])
kbs_train

,id,paragraph,question,choices,answer,question_plus
0,kbs-korean54-61,벌써 삼십 분이나 지났군. 출입문 위쪽에 붙은 낡은 벽시계가 여덟 시 십오 분을 가...,윗글에 대한 설명으로 가장 적절한 것은?,"['과거 회상을 통해 중심 사건을 제시하고 있다.', '특정 인물의 시각을 빌려 등...",2,NaN
1,kbs-korean54-62,벌써 삼십 분이나 지났군. 출입문 위쪽에 붙은 낡은 벽시계가 여덟 시 십오 분을 가...,밑줄 친 ㉠~㉤의 의미를 추론한 것으로 적절하지 않은 것은?,"['㉠눈: 계절적 배경이 드러남과 동시에 사람들의 고단한 삶을 상징하고 있다.', ...",4,NaN
2,kbs-korean54-63,벌써 삼십 분이나 지났군. 출입문 위쪽에 붙은 낡은 벽시계가 여덟 시 십오 분을 가...,윗글에 나타난 인물에 대한 설명으로 가장 적절한 것은?,"['‘중년 사내’는 ‘대학생’의 행동에 불쾌함을 느꼈다.', '‘역장’은 ‘오 씨 ...",5,NaN
3,kbs-korean54-64,까마득한 날에\n하늘이 처음 열리고\n어디 닭 우는 소리 들렸으랴.\n모든 산맥(山...,윗글에 대한 설명으로 적절하지 않은 것은?,"['시선의 이동에 따라 시상이 전개되고 있다.', '추상적인 대상을 구체화하여 표현...",1,NaN
4,kbs-korean54-65,까마득한 날에\n하늘이 처음 열리고\n어디 닭 우는 소리 들렸으랴.\n모든 산맥(山...,윗글에 대한 설명으로 적절하지 않은 것은?,['기쁨도 눈물이 없으면 기쁨이 아니다 사랑도 눈물 없는 사랑이 어디 있는가 - 정...,2,"속죄양 모티프는 인류의 죄를 대신해 십자가에 못 박힌 예수의 삶에서 비롯된 것으로,..."
...,...,...,...,...,...,...
2025,generation-for-nlp-2893,"“헐값에 팔리는 냉동 오렌지주스만 잔뜩 사가고, 쿠폰을 오려 모았으며, 구멍 난 스...",잭 맥도널드가 남긴 유산의 총액은 얼마인가?,"['1억8760만달러', '5000만달러', '2억달러', '1억달러', '3억달러']",1,NaN
2026,generation-for-nlp-2894,"넷기어코리아(지사장 김진겸, 이하 넷기어)가 뮤럴 캔버스를 가지고 넷기어 SNS에서...",넷기어가 모집하는 '뮤럴 공식 도슨트'의 주요 역할은 무엇인가?,"['전시물 설명 및 안내', '고객 서비스 제공', '제품 판매 촉진', '온라인 ...",4,NaN
2027,generation-for-nlp-2895,서울 성동구 옥수동과 금호동은 맞붙어 있는 동네지만 아파트값은 같은 면적에서 1억원...,옥수동과 금호동의 아파트값 차이가 발생한 주된 이유는 무엇인가?,"['옥수동의 재개발이 먼저 이루어졌다', '금호동의 인구가 더 많다', '옥수동이 ...",1,NaN
2028,generation-for-nlp-2896,방하남 고용노동부 장관(사진)이 취임 후 첫 외부 행사로 5년 만에 일자리를 3.5...,방하남 고용노동부 장관이 방문한 기업의 이름은 무엇인가?,"['셀트리온', '삼성전자', 'LG화학', '한화그룹', '현대자동차']",1,NaN


In [57]:
kbs_train.to_csv('kbs_auged_train_reformat.csv')